# Tutorial: Ethanol Electro-Oxydation Reaction

## Introduction

In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from melektrodica import *
directory = os.path.join("examples","SanchezMonreal2017Ethanol")
writer = Writer(log_file="SanchezMonreal2017Ethanol.log", log_directory=directory)
data = Collector(directory)
Kpy = Kpynetic(data)
melek = Calculator(Kpy)